In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import axes3d

In [2]:
import mdtraj
from sklearn.cluster import DBSCAN
from collections import Counter

In [3]:
xtc = mdtraj.formats.XTCTrajectoryFile("traj_120bar_640K.xtc")

In [4]:
def seek_start(xtc):
    """ Seeks 0 position of the trajectory. """
    xtc.seek(0)

In [5]:
def read_xyz(xtc, n=1):
    """ Reads only coordinates of n frames. """
    return xtc.read(n)[0][0]

In [6]:
xyz = read_xyz(xtc)
n_frames = len(xtc)
n_atoms = len(xyz)

In [7]:
eps = 0.0143
min_samples = 2
db = DBSCAN(eps, min_samples)
db.fit(xyz)

DBSCAN(algorithm='auto', eps=0.0143, leaf_size=30, metric='euclidean',
    min_samples=2, p=None, random_state=None)

In [8]:
def clusters_number(model):
    """ Returns number of clusters found in the model. """
    c = Counter(model.labels_)
    return len(c) - 1

In [9]:
def clusters_atoms_number(model):
    """ Returns number of atoms in clusters found in the model. """
    c = model.core_sample_indices_
    return len(c)

In [10]:
clusters_number(db)

0

In [11]:
clusters_atoms_number(db)

0

In [12]:
def set_axes(ax):
    """ Sets appropriate parameters of the axes """
    ax.patch.set_facecolor('white')
    ax.patch.set_alpha(0)
    ax.set_xlim3d([-5, 40])
    ax.set_ylim3d([-5, 40])
    ax.set_zlim3d([-5, 40])
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    return ax

In [13]:
def init(points):
    """ Animation initialization function.
        Seeks first frame.
    """
    points = ax.plot([], [], [],'go')[0]
    return points

In [14]:
def update(i, points, stat_text):
    """ Animation updating function.
        Reads next frame, finds clusters and updates coordinates of atoms.
    """
    xyz = read_xyz(xtc)
    db.fit(xyz)
    X = db.components_
    points.figure.suptitle('Frame %d of %d' %(i, n_frames))    
    stat_text.set_text('Number of clusters: %d\nNumber of atoms: %d' %(clusters_number(db),clusters_atoms_number(db)))
    points.set_data(X[:,0], X[:,1])
    points.set_3d_properties(X[:,2])
    return points,stat_text

In [15]:
def animate_clusters():
    """ Makes possible full animation.
        Initializes figure, axes, plot, text. Creates animation and runs it.
    """
    seek_start(xtc)
    fig = plt.figure(figsize=(10,4))
    ax = fig.add_subplot(121, projection='3d') # 3d-plot axes
    ax = set_axes(ax) # configure axes
    
    points = ax.plot([], [], [],'o')[0]
    
    params_text = ax.text2D(1.5, 0.7,'eps = %f,\nminsamples = %d' %(eps,min_samples),
                            transform=ax.transAxes) # parameters of the model
    stat_text = ax.text2D(1.5, 0.5, "Number of clusters: 0\nNumber of atoms: 0",
                          transform=ax.transAxes) # some statistics (quite primitive yet)
    
    anim = animation.FuncAnimation(fig, func=update,
                                   frames=n_frames-1, fargs=(points,stat_text), interval=20, blit=False)
    plt.tight_layout() # fits plot to screen
    plt.show()

In [ ]:
anim = animate_clusters()

In [ ]:
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=5, bitrate=1800)
# anim.save('anim_clusters.mp4', writer=writer)

In [ ]:
# анимация доступна по ссылке: https://drive.google.com/file/d/0B9n653m4f88BZUNzUElSOS1Uc1k/view?usp=sharing